<a href="https://colab.research.google.com/github/FilippoChen09/MNIST-CNN/blob/main/multiple-CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision

# 设置超参数
batch_size = 100
input_size = 784
hidden_size = 1000
num_classes = 10
num_epochs = 5
learning_rate = 0.001

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 从TorchVision下载MNIST数据集
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=torchvision.transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=torchvision.transforms.ToTensor())

# 使用PyTorch提供的DataLoader，以分批乱序加载数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

conv_layer_number = 1


class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv_start = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU()
        )
        # 卷积核大小为3，步长1，填充1则不改变图片尺寸，可直接叠加
        self.conv_loop = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU()
        )
        self.conv_end = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=16, out_channels=1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU()
        )

        self.fc = torch.nn.Linear(28 * 28, 10)

    def forward(self, x):
        x = self.conv_start(x)
        for i in range(conv_layer_number):
            x = self.conv_loop(x)
        x = self.conv_end(x)
        x = self.fc(x.reshape(-1, 28 * 28))
        return x


model = NeuralNetwork().to(device)
# 设置损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播算出Loss对各参数的梯度
        optimizer.zero_grad()
        loss.backward()

        # 更新参数
        optimizer.step()

# 检验模型在测试集上的准确性
correct = 0
total = 0
for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy on test_set: {} %'.format(100 * correct / total))

Accuracy on test_set: 95.46 %
